<font size="6">Calculate GHG Intensity per GDP, PPP </font>

<font size="4">Load Environment Variables</font>

In [1]:
from pcaf_env import *
from functools import reduce
import numpy as np

using connect string: trino://MichaelTiemannOSC@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev
create schema if not exists osc_datacommons_dev.mdt_sandbox


Open a Trino connection using JWT for authentication

In [2]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

('aicoe_osc_demo_results',)
('default',)
('demo_dv',)
('dera',)
('essd',)
('iceberg_demo',)
('information_schema',)
('ingest',)
('mdt_sandbox',)
('pcaf_sovereign_footprint',)
('rmi',)
('sandbox',)
('wri_gppd',)


In [3]:
# define source and destination tables
# LULUCF (Land Use, Land-Use Change and Forestry)

ingest_table = 'sf_unfccc_results'
src_table_1 = 'sf_unfccc_with_lulucf'
src_table_2 = 'sf_unfccc_without_lulucf'
src_table_3 = 'sf_wdi_gdp'
src_table_4 = 'sf_primap_hist_emissions'
src_country_table = 'sf_unfccc_countries'
models_table = 'pcaf_dbt_models'

In [4]:
########################
essd_schema = 'essd'
essd_src_table = 'gwp100_data'
#essd_src_table = 'ghg_data' 

In [5]:
### ESSD data is not needed at the moment (maybe in the future)

'''
sql=f"""
select iso as country_iso_code,year(year) as validity_date, co2/1000 as value,t.* from {ingest_catalog}.{essd_schema}.{essd_src_table} t """ 
df_essd = pd.read_sql(sql, engine)
#df1["units"] = "kt"
df_essd = df_essd.convert_dtypes()
df_essd[df_essd['country_iso_code'] == 'ZWE'][df_essd['validity_date'] ==2020]
rename_columns = {'value':'total'}
agg_columns = { 'value' : 'sum'}
columns_order = ['attribute','industry_code','country_iso_code','validity_date','total','total_units']
df_essd = df_essd.groupby(['country_iso_code','validity_date'],as_index=False).agg(agg_columns).rename(columns=rename_columns)
df_essd
'''



#df_essd.info(verbose=True)

'\nsql=f"""\nselect iso as country_iso_code,year(year) as validity_date, co2/1000 as value,t.* from {ingest_catalog}.{essd_schema}.{essd_src_table} t """ \ndf_essd = pd.read_sql(sql, engine)\n#df1["units"] = "kt"\ndf_essd = df_essd.convert_dtypes()\ndf_essd[df_essd[\'country_iso_code\'] == \'ZWE\'][df_essd[\'validity_date\'] ==2020]\nrename_columns = {\'value\':\'total\'}\nagg_columns = { \'value\' : \'sum\'}\ncolumns_order = [\'attribute\',\'industry_code\',\'country_iso_code\',\'validity_date\',\'total\',\'total_units\']\ndf_essd = df_essd.groupby([\'country_iso_code\',\'validity_date\'],as_index=False).agg(agg_columns).rename(columns=rename_columns)\ndf_essd\n'

In [7]:
sql=f"""
select country_iso_code,country_name,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_1}_source """ 
df1 = pd.read_sql(sql, engine)
#df1["units"] = "kt"
df1 = requantify_df(df1).convert_dtypes()
df1
df1.info(verbose=True)
sql=f"""
select t.country_iso_code,country_name,validity_date,attribute,value,value_units
from {ingest_catalog}.{ingest_schema}.{src_table_2}_source t
     join {ingest_catalog}.{ingest_schema}.{src_country_table}_source c on t.country_iso_code= c.country_iso_code 
where c.annex1_flag='Y'"""
df2 = pd.read_sql(sql, engine)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2256 entries, 0 to 2255
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype           
---  ------            --------------  -----           
 0   country_iso_code  2256 non-null   string          
 1   country_name      2256 non-null   string          
 2   validity_date     2256 non-null   Int64           
 3   attribute         2256 non-null   string          
 4   value             2256 non-null   pint[CO2eq * kt]
dtypes: Int64(1), pint[CO2eq * kt](1), string(3)
memory usage: 90.5 KB


In [8]:
#df2 = requantify_df(df2).convert_dtypes()
#df2
# gdp
sql=f"""
select country_iso_code,validity_date,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_3}_source
where attribute='GDP (current US$)' """
df3 = pd.read_sql(sql, engine)
df3
df3 = requantify_df(df3).convert_dtypes()

In [9]:
# gdp ppp
sql=f"""
select country_iso_code,validity_date,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_3}_source
where attribute='GDP, PPP (current international $)'"""
df4 = pd.read_sql(sql, engine)
df4 = requantify_df(df4).convert_dtypes()
df4

,country_iso_code,validity_date,value
0,ABW,2020,3329266442.16036
1,AFE,2020,2478218029367.22
2,AFG,2020,80911742458.6898
3,AFW,2020,1946025265851.92
4,AGO,2020,212918601685.596
...,...,...,...
7467,WSM,1991,374955750.72836
7468,YEM,1991,26589667519.9161
7469,ZAF,1991,266061272857.018
7470,ZMB,1991,12243272190.6853


In [11]:
# PRIMAP data

sql=f"""
select t.country_iso_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_4}_source t
       join {ingest_catalog}.{ingest_schema}.{src_country_table}_source c on t.country_iso_code=c.country_iso_code
where c.annex1_flag='N' and attribute='KYOTOGHG (AR4GWP100)' and validity_date>=1960  """ 
df_primap = pd.read_sql(sql, engine)
#df_primap['value_units'] = 'kt CO2'
#df_primap = requantify_df(df_primap).convert_dtypes()
# convert to CO2 units to t
#df_primap['value'] = df_primap['value'].pint.to("t CO2e")
df_primap

,country_iso_code,validity_date,attribute,value,value_units
0,SLB,1960,KYOTOGHG (AR4GWP100),36.7,CO2 * gigagram / a
1,SLB,1961,KYOTOGHG (AR4GWP100),51.0,CO2 * gigagram / a
2,SLB,1962,KYOTOGHG (AR4GWP100),51.9,CO2 * gigagram / a
3,SLB,1963,KYOTOGHG (AR4GWP100),55.1,CO2 * gigagram / a
4,SLB,1964,KYOTOGHG (AR4GWP100),60.2,CO2 * gigagram / a
...,...,...,...,...,...
9295,GNB,2017,KYOTOGHG (AR4GWP100),3000.0,CO2 * gigagram / a
9296,GNB,2018,KYOTOGHG (AR4GWP100),3170.0,CO2 * gigagram / a
9297,GNB,2019,KYOTOGHG (AR4GWP100),3190.0,CO2 * gigagram / a
9298,GNB,2020,KYOTOGHG (AR4GWP100),3260.0,CO2 * gigagram / a


In [12]:
#df2 = pd.merge(df2,df_essd,on=['country_iso_code','validity_date'],how='outer')  
#df2.rename(columns={"value": "ghg_total_without_lulucf", "total": "ghg_total_without_lulucf_essd"},inplace=True)
df2 = pd.merge(df2,df_primap,on=['country_iso_code','validity_date'],how='outer')  
df2.rename(columns={"value_units_x":"value_units","value_x": "ghg_total_without_lulucf_unfccc","value_y": "ghg_total_without_lulucf_primap"},inplace=True)
df2=df2.convert_dtypes()
df2.info(verbose=True)
df_primap.info(verbose=True)
df2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10633 entries, 0 to 10632
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   country_iso_code                 10633 non-null  string 
 1   country_name                     1333 non-null   string 
 2   validity_date                    10633 non-null  Int64  
 3   attribute_x                      1333 non-null   string 
 4   ghg_total_without_lulucf_unfccc  1333 non-null   Float64
 5   value_units                      1333 non-null   string 
 6   attribute_y                      9300 non-null   string 
 7   ghg_total_without_lulucf_primap  9300 non-null   Float64
 8   value_units_y                    9300 non-null   string 
dtypes: Float64(2), Int64(1), string(6)
memory usage: 778.9 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9300 entries, 0 to 9299
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dt

,country_iso_code,country_name,validity_date,attribute_x,ghg_total_without_lulucf_unfccc,value_units,attribute_y,ghg_total_without_lulucf_primap,value_units_y
0,AUT,Austria,1990,"Time Series - GHG total without LULUCF, in kt ...",78423.22073,kt CO2e,<NA>,<NA>,<NA>
1,AUT,Austria,1991,"Time Series - GHG total without LULUCF, in kt ...",82095.386494,kt CO2e,<NA>,<NA>,<NA>
2,AUT,Austria,1992,"Time Series - GHG total without LULUCF, in kt ...",75464.121664,kt CO2e,<NA>,<NA>,<NA>
3,AUT,Austria,1993,"Time Series - GHG total without LULUCF, in kt ...",75806.345879,kt CO2e,<NA>,<NA>,<NA>
4,AUT,Austria,1994,"Time Series - GHG total without LULUCF, in kt ...",76039.471735,kt CO2e,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...
10628,GNB,<NA>,2017,<NA>,<NA>,<NA>,KYOTOGHG (AR4GWP100),3000.0,CO2 * gigagram / a
10629,GNB,<NA>,2018,<NA>,<NA>,<NA>,KYOTOGHG (AR4GWP100),3170.0,CO2 * gigagram / a
10630,GNB,<NA>,2019,<NA>,<NA>,<NA>,KYOTOGHG (AR4GWP100),3190.0,CO2 * gigagram / a
10631,GNB,<NA>,2020,<NA>,<NA>,<NA>,KYOTOGHG (AR4GWP100),3260.0,CO2 * gigagram / a


In [13]:
df2['scope1_excl_source'] = np.where(df2.ghg_total_without_lulucf_unfccc.notnull(),
         'UNFCCC',
         np.where(df2.ghg_total_without_lulucf_primap.notnull(),
                  'PRIMAP','UNFCCC'))
#df2['scope1_source'] = df2['ghg_total_without_lulucf'].apply(lambda x: 'UNFCCC' if x.isnull() == True else 'PRIMAP')
df2["value"] = df2["ghg_total_without_lulucf_unfccc"].fillna(df2["ghg_total_without_lulucf_primap"])
df2.drop(columns=["attribute_x","attribute_y","value_units_y","ghg_total_without_lulucf_unfccc","ghg_total_without_lulucf_primap"],inplace=True)

columns_order = ['country_iso_code','country_name','validity_date','value','value_units','scope1_excl_source']
df2 = df2.reindex(columns=columns_order).convert_dtypes()
df2.info(verbose=True)
##df2[df2["scope1_excl_source"] =='PRIMAP']
#df2 = requantify_df(df2)

#df2

#df2.sort_values(by=['country_iso_code','validity_date'], ascending=False).to_csv("tmp_scope1_waterfall.csv")
df2[df2["country_iso_code"] == 'CAN']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10633 entries, 0 to 10632
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country_iso_code    10633 non-null  string 
 1   country_name        1333 non-null   string 
 2   validity_date       10633 non-null  Int64  
 3   value               10633 non-null  Float64
 4   value_units         1333 non-null   string 
 5   scope1_excl_source  10633 non-null  string 
dtypes: Float64(1), Int64(1), string(4)
memory usage: 519.3 KB


,country_iso_code,country_name,validity_date,value,value_units,scope1_excl_source
248,CAN,Canada,1990,594722.24342,kt CO2e,UNFCCC
249,CAN,Canada,1991,587905.793885,kt CO2e,UNFCCC
250,CAN,Canada,1992,605290.24882,kt CO2e,UNFCCC
251,CAN,Canada,1993,607681.076914,kt CO2e,UNFCCC
252,CAN,Canada,1994,628145.869194,kt CO2e,UNFCCC
253,CAN,Canada,1995,645455.296776,kt CO2e,UNFCCC
254,CAN,Canada,1996,666701.91023,kt CO2e,UNFCCC
255,CAN,Canada,1997,682298.998433,kt CO2e,UNFCCC
256,CAN,Canada,1998,688613.931025,kt CO2e,UNFCCC
257,CAN,Canada,1999,701758.711729,kt CO2e,UNFCCC


Read the source tables into dataframes  

<font size="4">Merge the source dataframes in a single dataframe and calculate ghg intensity values</font>

In [15]:
df2 = requantify_df(df2.fillna(value={'value': 0}))
df2

/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:227: RuntimeWarning: pint-pandas does not support magnitudes of object. Converting magnitudes to float.
  warnings.warn(


,country_iso_code,country_name,validity_date,value,scope1_excl_source
0,AUT,Austria,1990,78423.2207298205,UNFCCC
1,AUT,Austria,1991,82095.38649411147,UNFCCC
2,AUT,Austria,1992,75464.12166419617,UNFCCC
3,AUT,Austria,1993,75806.34587896381,UNFCCC
4,AUT,Austria,1994,76039.4717350331,UNFCCC
...,...,...,...,...,...
10628,GNB,<NA>,2017,3000.0,PRIMAP
10629,GNB,<NA>,2018,3170.0,PRIMAP
10630,GNB,<NA>,2019,3190.0,PRIMAP
10631,GNB,<NA>,2020,3260.0,PRIMAP


In [16]:

df_result = pd.merge(df1,df2,on=['country_iso_code','validity_date'],how='outer')  
df_result=df_result.convert_dtypes()
df_result.drop(columns=['country_name_y','attribute'],inplace=True)
df_result.rename(columns={"country_name_x":"country_name", "value_x":"ghg_total_with_lulucf", "value_y":"ghg_total_without_lulucf"},inplace=True)

###

df_result = pd.merge(df_result,df3,on=['country_iso_code','validity_date'],how="outer")  
df_result.rename(columns={"value":"gdp"},inplace=True)
###
df_result = pd.merge(df_result,df4,on=['country_iso_code','validity_date'],how="outer")  
df_result.rename(columns={"value":"gdp_ppp"},inplace=True)



df_result["ghg_intensity_with_lulucf_per_gdp"] = df_result["ghg_total_with_lulucf"]/df_result["gdp_ppp"]*1000000
df_result["ghg_intensity_without_lulucf_per_gdp"] = df_result["ghg_total_without_lulucf"]/df_result["gdp_ppp"]*1000000
df_result.info(verbose=True)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15025 entries, 0 to 15024
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype                 
---  ------                                --------------  -----                 
 0   country_iso_code                      15025 non-null  string                
 1   country_name                          2256 non-null   string                
 2   validity_date                         15025 non-null  Int64                 
 3   ghg_total_with_lulucf                 2256 non-null   pint[CO2eq * kt]      
 4   ghg_total_without_lulucf              10633 non-null  pint[CO2eq * kt]      
 5   scope1_excl_source                    10633 non-null  string                
 6   gdp                                   13118 non-null  pint[USD]             
 7   gdp_ppp                               7472 non-null   pint[USD]             
 8   ghg_intensity_with_lulucf_per_gdp     2090 non-null   pint[CO2eq *

In [17]:
df1 = dequantify_df(df1)
df2 = dequantify_df(df2)
df3 = dequantify_df(df3)
df_result = dequantify_df(df_result)
df_result=df_result.convert_dtypes()
df_result.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15025 entries, 0 to 15024
Data columns (total 16 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   country_iso_code                            15025 non-null  string 
 1   country_name                                2256 non-null   string 
 2   validity_date                               15025 non-null  Int64  
 3   ghg_total_with_lulucf                       2256 non-null   Float64
 4   ghg_total_with_lulucf_units                 15025 non-null  string 
 5   ghg_total_without_lulucf                    10633 non-null  Float64
 6   ghg_total_without_lulucf_units              15025 non-null  string 
 7   scope1_excl_source                          10633 non-null  string 
 8   gdp                                         13118 non-null  Float64
 9   gdp_units                                   15025 non-null  string 
 10  gdp_ppp   

Run these in a notebook cell if you need to install onto your nb env

<font size="5">Save the results in Trino</font>


In [6]:
try:
    os.mkdir("/opt/app-root/src/PCAF-sovereign-footprint/dbt/pcaf_transform/models", mode=0o755)
except FileExistsError:
    pass
shutil.rmtree("/opt/app-root/src/PCAF-sovereign-footprint/dbt/pcaf_transform/models/.ipynb_checkpoints", ignore_errors=True)

In [13]:
custom_meta_content = {
    'data provider': 'UNFCCC',
    'title': 'UNFCCC sovereign emissions by country and year',
    'author': 'UNFCCC',
    'contact': 'GHGdata@unfccc.int',
    'description': json.dumps("""
UNFCCC sovereign emissions by country and year"""),
    'release_date': '2022-01-01 00:00:00',
    # How should we describe our transformative step here?
}

def description_is (s):
    return { 'Description': json.dumps(s)}

custom_meta_fields = {}
custom_meta_fields['country_iso_code'] = description_is("ISO-3166 Country Code (alpha_3)")
custom_meta_fields['country_name'] = description_is("ISO-3166 Country Name")
custom_meta_fields['validity_date'] = description_is("Year of measurement")
custom_meta_fields['ghg_total_with_lulucf'] = description_is("if not NULL, GHG total including LULUCF")
custom_meta_fields['ghg_total_with_lulucf_units'] = description_is("kt CO2eq")
custom_meta_fields['ghg_total_without_lulucf'] = description_is("if not NULL, GHG total excluding LULUCF")
custom_meta_fields['ghg_total_without_lulucf_units'] = description_is("kt CO2eq")
custom_meta_fields['scope1_excl_source'] = description_is("UNFCC or PRIMAP or NULL")
custom_meta_fields['gdp'] = description_is("Gross Domestic Product")
custom_meta_fields['gdp_units'] = description_is("USD")
custom_meta_fields['gdp_ppp'] = description_is("Gross Domestic Product, normalized to Purchasing Power Parity")
custom_meta_fields['gdp_ppp_units'] = description_is("USD")
custom_meta_fields['ghg_intensity_with_lulucf_per_gdp'] = description_is("GHG tons including LULUCH divided by GDP")
custom_meta_fields['ghg_intensity_with_lulucf_per_gdp_units'] = description_is("kt CO2e / USD")
custom_meta_fields['ghg_intensity_without_lulucf_per_gdp'] = description_is("GHG tons excluding LULUCH divided by GDP")
custom_meta_fields['ghg_intensity_without_lulucf_per_gdp_units'] = description_is("kt CO2e / USD")

for f in custom_meta_fields:
    if 'ISO' in custom_meta_fields[f]['Description']:
        custom_meta_fields[f]['tags'] = ['ISO']
    elif 'IPCC' in custom_meta_fields[f]['Description']:
        custom_meta_fields[f]['tags'] = ['IPCC']
    elif 'WDI' in custom_meta_fields[f]['Description']:
        custom_meta_fields[f]['tags'] = ['WDI']
    elif f == 'year':
        custom_meta_fields[f]['tags'] = ['annual']

rec_source
data_provider
country_iso_code
country_name
attribute
value
value_units
validity_date


In [19]:
create_trino_table_and_dbt_metadata(ingest_table, df_result, ['country_iso_code'], custom_meta_content='', custom_meta_fields='', verbose=True)

drop table if exists mdt_sandbox.sf_unfccc_results_source
enforcing dataframe partition column order

verifying existence of table osc_datacommons_dev.mdt_sandbox.sf_unfccc_results_source
create table if not exists osc_datacommons_dev.mdt_sandbox.sf_unfccc_results_source (
    country_name varchar,
    validity_date bigint,
    ghg_total_with_lulucf double,
    ghg_total_with_lulucf_units varchar,
    ghg_total_without_lulucf double,
    ghg_total_without_lulucf_units varchar,
    scope1_excl_source varchar,
    gdp double,
    gdp_units varchar,
    gdp_ppp double,
    gdp_ppp_units varchar,
    ghg_intensity_with_lulucf_per_gdp double,
    ghg_intensity_with_lulucf_per_gdp_units varchar,
    ghg_intensity_without_lulucf_per_gdp double,
    ghg_intensity_without_lulucf_per_gdp_units varchar,
    country_iso_code varchar
) with (
    format = 'parquet',
    partitioning = array['country_iso_code']
)

staging dataframe parquet to s3 osc-datacommons-s3-bucket-dev02
/tmp/ingest_temp_68551

In [20]:
#df_result.to_csv(ingest_table+".csv")
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}_source where country_iso_code='TWN' and validity_date > 2015"""
pd.read_sql(sql, engine)


,country_name,validity_date,ghg_total_with_lulucf,ghg_total_with_lulucf_units,ghg_total_without_lulucf,ghg_total_without_lulucf_units,scope1_excl_source,gdp,gdp_units,gdp_ppp,gdp_ppp_units,ghg_intensity_with_lulucf_per_gdp,ghg_intensity_with_lulucf_per_gdp_units,ghg_intensity_without_lulucf_per_gdp,ghg_intensity_without_lulucf_per_gdp_units,country_iso_code
0,None,2016,None,CO2eq * kt,298000.0,CO2eq * kt,PRIMAP,None,USD,None,USD,None,CO2eq * kt / USD,None,CO2eq * kt / USD,TWN
1,None,2017,None,CO2eq * kt,304000.0,CO2eq * kt,PRIMAP,None,USD,None,USD,None,CO2eq * kt / USD,None,CO2eq * kt / USD,TWN
2,None,2018,None,CO2eq * kt,302000.0,CO2eq * kt,PRIMAP,None,USD,None,USD,None,CO2eq * kt / USD,None,CO2eq * kt / USD,TWN
3,None,2019,None,CO2eq * kt,293000.0,CO2eq * kt,PRIMAP,None,USD,None,USD,None,CO2eq * kt / USD,None,CO2eq * kt / USD,TWN
4,None,2020,None,CO2eq * kt,291000.0,CO2eq * kt,PRIMAP,None,USD,None,USD,None,CO2eq * kt / USD,None,CO2eq * kt / USD,TWN
5,None,2021,None,CO2eq * kt,305000.0,CO2eq * kt,PRIMAP,None,USD,None,USD,None,CO2eq * kt / USD,None,CO2eq * kt / USD,TWN


In [16]:
osc._do_sql(f"insert into {ingest_schema}.{models_table} values(\'{json.dumps(dbt_dict['models'])}\')", engine, verbose=True)

insert into mdt_sandbox.pcaf_dbt_models values('{"sf_wdi_gdp": {"description": "\"\\nThis indicator provides per capita values for gross domestic product (GDP) expressed in current international dollars converted by purchasing power parity (PPP) conversion factor. \\n\\nGDP is the sum of gross value added by all resident producers in the country plus any product taxes and minus any subsidies not included in the value of the products. conversion factor is a spatial price deflator and currency converter that controls for price level differences between countries. Total population is a mid-year population based on the de facto definition of population, which counts all residents regardless of legal status or citizenship.\"", "columns": {"rec_source": {"description": "\"API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv and API_NY.GDP.MKTP.PP.CD_DS2_en_csv_v2.csv\""}, "data_provider": {"description": "\"WDI\"", "tags": ["WDI"]}, "country_iso_code": {"description": "\"ISO-3166 Country Code (alpha_3)\"", 

[(1,)]